# Project 5: Prompting With Large Language Models

In this project, we learn how to solve tasks by prompting existing LLM APIs. We will experiment with zero-shot and few-shot prompting and different ways methods for example selection for a semantic parsing task.

First we install and import the required dependencies. These include:
* `openai` as our API for querying LLMs (you are free to choose to use a different LLM API if you would like)


In [1]:
%%capture
%pip install openai

If you are using the OpenAI API, then go to then create an account and then copy your secret API key from `https://platform.openai.com/account/api-keys`. Set this as an environment variable or key management service so we can load it below. Make to keep private key secret. You may use a different LLM service if you choose to eg. Cohere (https://cohere.ai/).

In [2]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

If you have successfully authorized, then you should be able to see a list of available models by running the command below.

In [3]:
import openai
openai.api_key = "KEY"
openai.Model.list()

<OpenAIObject list at 0x7f2ac374f290> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

We will now evaluate the LLM on a semantic parsing task. Geoquery is a dataset that contains information about the geography of the United States. For more information, please see: https://www.cs.utexas.edu/users/ml/nldata/geoquery.html. We will experiment with the compositional split introduced in (Keysers et al., 2020) https://openreview.net/forum?id=SygcCnNKwr. First, let's download the train and validation data. The goal for the LLM is to take in English queries about US geography about population, elevation, etc. and output a formal representation of the query.

In [4]:
!wget https://github.com/kl2806/geoquery/raw/main/data.zip -O data.zip
!unzip -o data.zip

--2023-05-02 01:06:11--  https://github.com/kl2806/geoquery/raw/main/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip [following]
--2023-05-02 01:06:11--  https://raw.githubusercontent.com/kl2806/geoquery/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4493 (4.4K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   4.39K  --.-KB/s    in 0s      

2023-05-02 01:06:12 (46.1 MB/s) - ‘data.zip’ saved [4493/4493]

Archive:  data.zip
  inflating: train.tsv               
  inflating: dev.tsv                 


Let's take a look at the data, each instance should compose of an English utterance, and a formal representation of the utterance

In [5]:
!head -n 5 ./train.tsv

how tall is the highest point in m0	answer ( elevation_1 ( highest ( intersection ( place , loc_2 ( m0 ) ) ) ) )
what is the largest city in m0	answer ( largest ( intersection ( city , loc_2 ( m0 ) ) ) )
what states border states that the m0 runs through	answer ( intersection ( state , next_to_2 ( intersection ( state , traverse_1 ( m0 ) ) ) ) )
what is the maximum elevation of m0	answer ( highest ( intersection ( place , loc_2 ( m0 ) ) ) )
what is the population of m0	answer ( population_1 ( m0 ) )


In [51]:
import csv
from dataclasses import dataclass
from typing import List

@dataclass
class Example:
     query: str
     program: str

training_examples: List[Example] = []

with open('./train.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        training_examples.append(Example(query, program))

dev_examples: List[Example] = []
with open('./dev.tsv', 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for query, program in reader:
        dev_examples.append(Example(query, program))

print(f"Num training examples: {len(training_examples)}")
print(f"Num dev examples: {len(dev_examples)}")


Num training examples: 440
Num dev examples: 40


In [7]:
print(training_examples[:5])

[Example(query='how tall is the highest point in m0', program='answer ( elevation_1 ( highest ( intersection ( place , loc_2 ( m0 ) ) ) ) )'), Example(query='what is the largest city in m0', program='answer ( largest ( intersection ( city , loc_2 ( m0 ) ) ) )'), Example(query='what states border states that the m0 runs through', program='answer ( intersection ( state , next_to_2 ( intersection ( state , traverse_1 ( m0 ) ) ) ) )'), Example(query='what is the maximum elevation of m0', program='answer ( highest ( intersection ( place , loc_2 ( m0 ) ) ) )'), Example(query='what is the population of m0', program='answer ( population_1 ( m0 ) )')]


Now, let's define a function that uses the OpenAI API to output the semantic parse. As a first cut, let's just try to describe the task in English the model and return it in `get_static_prompt`. Then implement `parse_example`, which should call the LLM API, and return the semantic parse. `gpt-3.5-turbo`, the corresponding API call to ChatGPT should work well for this task.

In [8]:
from typing import Callable
import numpy as np

def get_static_prompt(utterance: str) -> str:
    """Return a prompt that doesn't change between different examples"""
    return f"Prompt: {utterance}\nResponse: "
    
def parse_example(model: str, utterance: str, prompt_method: Callable[[str], str], **kwargs: dict) -> str:
    """Return the semantic parse of the utterance"""
    prompt = prompt_method(utterance, **kwargs)
    """YOUR CODE HERE"""
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": prompt}],
    )
    return response

parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=get_static_prompt)
print(parse)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I'm sorry, but I cannot answer that prompt without more information. Can you please provide me with the location of m0?",
        "role": "assistant"
      }
    }
  ],
  "created": 1682989579,
  "id": "chatcmpl-7BYl9H2GDJbacwv42KKppBjxBfzlh",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 20,
    "total_tokens": 46
  }
}


With just an English description, the output probably does not look very similar to the target language that we want. Let's try to construct a prompt with some examples from our training set and see how it does. Implement the function below for uniformly sampling examples from the training set, and using that to as a few-shot prompt to the model.

In [9]:
import random
from typing import List
import numpy as np

def random_sample_prompt(utterance: str, training_examples: List[Example], num_samples: int = 10) -> str:
    """Return a prompt for a given example"""
    """YOUR CODE HERE"""
    random = np.random.randint(0, len(training_examples), (num_samples, ))
    sample_prompt = ""
    for idx in random:
        prompt = get_static_prompt(training_examples[idx].query) + training_examples[idx].program + "\n"
        sample_prompt += prompt
    sample_prompt += get_static_prompt(utterance)
    return sample_prompt
    
prompt = random_sample_prompt(utterance="what river runs through m0",
                              training_examples=training_examples)
print("Uniform random sampling prompt: \n", prompt)


Uniform random sampling prompt: 
 Prompt: how many people live in m0
Response: answer ( population_1 ( m0 ) )
Prompt: how many people in m0
Response: answer ( population_1 ( m0 ) )
Prompt: what is the smallest city in m0
Response: answer ( smallest ( intersection ( city , loc_2 ( m0 ) ) ) )
Prompt: how many people live in m0
Response: answer ( population_1 ( m0 ) )
Prompt: what is the population of the capital of the largest state
Response: answer ( population_1 ( intersection ( capital , loc_2 ( largest ( state ) ) ) ) )
Prompt: what is the adjacent state of m0
Response: answer ( intersection ( state , next_to_2 ( m0 ) ) )
Prompt: what are the cities in m0
Response: answer ( intersection ( city , loc_2 ( m0 ) ) )
Prompt: how many rivers are in m0
Response: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Prompt: what is the area of m0
Response: answer ( area_1 ( m0 ) )
Prompt: how high is the highest point of m0
Response: answer ( elevation_1 ( highest ( intersection ( place

In [10]:
parse = parse_example(model="gpt-3.5-turbo",
                      utterance="what river runs through m0",
                      prompt_method=random_sample_prompt,
                      training_examples=training_examples)
print(parse)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "answer ( intersection ( river , loc_2 ( m0 ) ) )",
        "role": "assistant"
      }
    }
  ],
  "created": 1682989587,
  "id": "chatcmpl-7BYlHIK2Y6eUg0AP8bNkfBWhsQFiu",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 329,
    "total_tokens": 344
  }
}


Now, let's evaluate our uniform sampling prompt on the validation set. If you run into rate limit issues with the API, you may want to backoff or consult one of the solutions here https://platform.openai.com/docs/guides/rate-limits/error-mitigation.

In [24]:
import tqdm

def get_predictions(model: str,
                    evaluation_examples: List[Example],
                    prompt_creation_function: Callable[[str], str],
                    **kwargv: List[str]) -> List[str]:
    """Get a list of predictions from the evaluation examples"""
    predictions = []
    for example in tqdm.tqdm(evaluation_examples):
        predicted_program = parse_example(model, example.query, prompt_creation_function, **kwargv)
        predictions.append(predicted_program)
    return predictions

def evaluate(predictions: List[str], evaluation_examples: List[Example]) -> float:
    """Evaluate the accuracy of the predictions"""
    correct = 0
    for prediction, example in zip(predictions, evaluation_examples):
        if prediction['choices'][0]['message']['content'] == example.program:
            correct += 1
    return correct / len(evaluation_examples)

In [12]:
random_sample_predictions = get_predictions(model="gpt-3.5-turbo",
                                            evaluation_examples=dev_examples,
                                            prompt_creation_function=random_sample_prompt,
                                            training_examples=training_examples)


100%|██████████| 40/40 [00:55<00:00,  1.38s/it]


The model should get at least 15\% exact match with randomly sampling examples.

In [25]:
# Save the predictions as `random_predictions.txt`
with open('random_predictions.txt', 'w') as f:
    for prediction in random_sample_predictions:
        f.write(prediction['choices'][0]['message']['content'])
        f.write('\n')

exact_match = evaluate(random_sample_predictions, dev_examples)
print(f"Exact match for uniform sampling prompt: {exact_match}")

Exact match for uniform sampling prompt: 0.375


Randomly sampling examples does not use consider the utterance when selecting examples. Next, we will try to pick examples for the prompt based on embedding similarity. First, let's install `sentence-transformers`, which we will use to get embeddings of the utterance.

In [26]:
%%capture
%pip install -U sentence-transformers

Now, let's construct embeddings using a small pretrained model for each example in our training data.

In [35]:

from sentence_transformers import SentenceTransformer, util
import torch

def get_corpus(model_name: str, examples: List[Example]) -> torch.Tensor:
    """Return a tensor of the corpus embeddings, the size of the returned tensor should be (num_examples, embedding_size)"""
    """YOUR CODE HERE"""
    sentences = [ex.query for ex in examples]
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences)
    return torch.tensor(embeddings).type(torch.float32)

corpus_embeddings = get_corpus('all-MiniLM-L6-v2', training_examples)

In [36]:
print(type(corpus_embeddings))

<class 'torch.Tensor'>


In [53]:
training_examples[106]

Example(query='what rivers are in m0', program='answer ( intersection ( river , loc_2 ( m0 ) ) )')

With the embeddings of all the training data, we can construct a function that takes in an utterance and outputs a prompt with examples with highest cosine similarity with the utterance.

In [55]:
def get_nearest_neighbor_prompt(utterance: str,
                                training_examples: List[Example],
                                embedding_model: str,
                                corpus_embeddings: torch.Tensor,
                                num_samples: int=10) -> str:
    """YOUR CODE HERE"""
    model = SentenceTransformer(embedding_model)
    embeddings = model.encode([utterance])
    sims = util.cos_sim(embeddings, corpus_embeddings).squeeze()
    top_samples = torch.topk(sims, k=num_samples, largest=True).indices
    sample_prompt = ""
    for idx in top_samples:
        prompt = get_static_prompt(training_examples[idx].query) + training_examples[idx].program + "\n"
        sample_prompt += prompt

    sample_prompt += get_static_prompt(utterance)
    return sample_prompt

Evaluate the similarity based on prompt on the validation data and save your predictions as `similarity_predictions.txt` with one predictions per line of the validation set. With similarity based example selection, we should get at least 20\% exact match on the validation set. Note that there could be some duplicates in the training data because we are working with a version of the data where the location names are normalized to be variables like `m0`.

In [56]:
similarity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=get_nearest_neighbor_prompt,
                                         training_examples=training_examples,
                                         embedding_model='all-MiniLM-L6-v2',
                                         corpus_embeddings=corpus_embeddings)

# Save the predictions as `similarity_predictions.txt`
with open('similarity_predictions.txt', 'w') as f:
    for prediction in similarity_predictions:
        f.write(prediction['choices'][0]['message']['content'])
        f.write('\n')

exact_match = evaluate(similarity_predictions, dev_examples)
print(f"Exact match for nearest neighbor prompt: {exact_match}")

100%|██████████| 40/40 [01:17<00:00,  1.93s/it]

Exact match for nearest neighbor prompt: 0.525


Let's try to improve on nearest neighbor based example search. This part will be more open ended. We will now implement a different example selection method that improves over uniform random selection. You may implement an algorithm `Diverse Demonstrations Improve In-context Compositional Generalization` (https://arxiv.org/abs/2212.06800) or come up with your own example selection method. In the report, describe the algorithm that you implemented and intuitiion of why it may it could be effective.

In [85]:
def construct_diversity_prompt(utterance: str,
                               training_examples: List[Example],
                               num_samples: int = 10) -> str:
    """YOUR CODE HERE"""
    sentences = [ex.query for ex in training_examples]
    sentences = []
    resp = []
    str_set = set()
    for ex in training_examples:
        if ex.query in str_set:
            continue
        sentences.append(ex.query)
        resp.append(ex.program)
        str_set.add(ex.query)

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences)
    corpus_embeddings = torch.tensor(embeddings).type(torch.float32)
    embeddings = model.encode([utterance])
    sims = util.cos_sim(embeddings, corpus_embeddings).squeeze()
    top_samples = torch.topk(sims, k=num_samples, largest=True).indices
    sample_prompt = ""
    for idx in top_samples:
        prompt = get_static_prompt(sentences[idx]) + resp[idx] + "\n"
        sample_prompt += prompt

    sample_prompt += get_static_prompt(utterance)
    return sample_prompt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [86]:
print(construct_diversity_prompt(utterance="what river runs through m0", training_examples=training_examples))


Prompt: which rivers are in m0
Response: answer ( intersection ( river , loc_2 ( m0 ) ) )
Prompt: what rivers are in m0
Response: answer ( intersection ( river , loc_2 ( m0 ) ) )
Prompt: how many rivers are in m0
Response: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Prompt: what are all the rivers in m0
Response: answer ( intersection ( river , loc_2 ( m0 ) ) )
Prompt: how many rivers are there in m0
Response: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Prompt: how many rivers does m0 have
Response: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Prompt: how many rivers in m0
Response: answer ( count ( intersection ( river , loc_2 ( m0 ) ) ) )
Prompt: which states does the m0 river run through
Response: answer ( intersection ( state , traverse_1 ( intersection ( river , m0 ) ) ) )
Prompt: what states does the m0 river run through
Response: answer ( intersection ( state , traverse_1 ( intersection ( river , m0 ) ) ) )
Prompt: what is the biggest 

In [87]:
diversity_predictions = get_predictions(model="gpt-3.5-turbo",
                                         evaluation_examples=dev_examples,
                                         prompt_creation_function=construct_diversity_prompt,
                                         training_examples=training_examples)

# Save the predictions as `diversity_predictions.txt`
with open('diversity_predictions.txt', 'w') as f:
    for prediction in diversity_predictions:
        f.write(prediction['choices'][0]['message']['content'])
        f.write('\n')

exact_match = evaluate(diversity_predictions, dev_examples)
print(f"Exact match for diversity based prompt: {exact_match}")

100%|██████████| 40/40 [02:20<00:00,  3.50s/it]

Exact match for diversity based prompt: 0.65


Get the predictions and submit these as `diversity_predictions.txt`, where each line is a prediction for the development set. With the improved selection, we should get least 35\% exact match on the validation set.

For the report, compare the predictions from the example selection methods using 1) uniform random sampling 2) embedding-based similarity search and 3) coverage based selection. Compare and contrast the errors and submit your analysis as `report.pdf`

* hw5.ipynb (this file; please rename to match)
* random_predictions.txt
* similarity_predictions.txt
* diversity_predictions.txt
* report.pdf